# Armani SageMaker instance 

In [16]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [17]:
bucket_name = 'cestino-s3'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


The part until now is equal to the [tutorial](https://aws.amazon.com/it/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/).

In [18]:
try:
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv", "./products.csv")
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./products.csv', header=None, index_col=False) # Changed this
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


Now that we have loaded into model_data the CSV...

In [19]:
print(model_data) # ...and after a bit of debugging...
print(type(model_data)) # ...and compensation of the absurd leak of documentation...

       0          1                         2                            3    
0       0          1                         2                            3  \
1   donna    giacche                       blu                         lino   
2   donna  pantaloni  grigioscuro-ghiaccio-blu                         lino   
3   donna  pantaloni                  fantasia                twill-di-seta   
4   donna   sneakers                bianco-blu                        pelle   
5   donna      abiti                  nero-blu                       jersey   
6   donna    giacche                  fantasia                twill-di-seta   
7   donna    sandali               argento-oro                        pelle   
8   donna    sandali                      nero                      vernice   
9   donna    sandali              nero-marrone                        nappa   
10  donna    sandali                      nero                 raso-sautage   
11  donna    giacche                   blunavy      

...we start splitting data into the data for training (in this case 70% of the total) an data for testing (the rest).

In [20]:
# train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
train_data, test_data = model_data, model_data
print(train_data.shape, test_data.shape)

(51, 12) (51, 12)


We should first get what [random_state](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) is.
Well, it's just a randomizer I guess.

In [21]:
print(train_data)
print(test_data)

       0          1                         2                            3    
0       0          1                         2                            3  \
1   donna    giacche                       blu                         lino   
2   donna  pantaloni  grigioscuro-ghiaccio-blu                         lino   
3   donna  pantaloni                  fantasia                twill-di-seta   
4   donna   sneakers                bianco-blu                        pelle   
5   donna      abiti                  nero-blu                       jersey   
6   donna    giacche                  fantasia                twill-di-seta   
7   donna    sandali               argento-oro                        pelle   
8   donna    sandali                      nero                      vernice   
9   donna    sandali              nero-marrone                        nappa   
10  donna    sandali                      nero                 raso-sautage   
11  donna    giacche                   blunavy      

In [22]:
train_data.to_csv(
  'train.csv',
  index=False,
  header=False
)
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(
        prefix, 'train/train.csv'
    )
).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [23]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(
    xgboost_container,role, instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=f's3://{bucket_name}/{prefix}/output',
    sagemaker_session=sess
)
xgb.set_hyperparameters(
    max_depth=5,eta=0.2,gamma=4,
    min_child_weight=6,subsample=0.8,
    silent=0,objective='binary:logistic',
    num_round=100
)

In [24]:
print(s3_input_train)

In [25]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-23-15-04-33-107


2023-06-23 15:04:33 Starting - Starting the training job...
2023-06-23 15:04:49 Starting - Preparing the instances for training......
2023-06-23 15:05:54 Downloading - Downloading input data...
2023-06-23 15:06:29 Training - Downloading the training image...
2023-06-23 15:07:00 Uploading - Uploading generated training modelArguments: train
[2023-06-23:15:06:53:INFO] Running standalone xgboost training.
[2023-06-23:15:06:53:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-06-23:15:06:53:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 55606.23mb
[2023-06-23:15:06:53:INFO] Determined delimiter of CSV input is ','
[15:06:53] S3DistributionType set as FullyReplicated
[15:06:53] 51x11 matrix with 561 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[15:06:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[0]#011train-error:0
[15:06:53] src/tree/up

Then we'll instanciate the sagemaker.predictor.Predictor...

In [ ]:
predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium'
)

INFO:sagemaker:Creating model with name: xgboost-2023-06-23-15-11-40-216
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-06-23-15-11-40-216
INFO:sagemaker:Creating endpoint with name xgboost-2023-06-23-15-11-40-216


--------------------------------------------------------------------------------------------------------------------------

In [ ]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop([11, 10], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)